In [ ]:
import numpy as np
import scipy.signal
import scipy.io.wavfile as wavfile
import pyaudio

# Function to create a Butterworth low-pass filter
def butter_lowpass(cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = scipy.signal.butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

# Function to apply the Butterworth filter to real-time data
def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = scipy.signal.lfilter(b, a, data)
    return y

# Parameters
cutoff_freq = 1000
fs = 44100
order = 5

# Set up PyAudio
p = pyaudio.PyAudio()

# Open a stream
stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=fs,
                input=True,
                output=True,
                frames_per_buffer=1024)

print("Filtering in real-time. Press Ctrl+C to exit.")

try:
    while True:
        # Read audio data from the stream
        input_data = stream.read(1024)
        input_array = np.frombuffer(input_data, dtype=np.int16)

        # Apply the filter
        filtered_array = butter_lowpass_filter(input_array, cutoff_freq, fs, order=order)

        # Convert the filtered array back to bytes
        output_data = filtered_array.astype(np.int16).tobytes()

        # Write the filtered data back to the stream
        stream.write(output_data)

except KeyboardInterrupt:
    print("Exiting...")

finally:
    # Close the stream and terminate PyAudio
    stream.stop_stream()
    stream.close()
    p.terminate()
